ใน ep นี้ เราจะ

# 0. Install

Install Library ที่จำเป็น

In [131]:
## Colab
! curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

Import Library ที่จะใช้ ในที่นี้คือ fastai.text และ sklearn_text ที่ใช้ในงาน [NLP](https://www.bualabs.com/archives/119/what-is-nlp-natural-language-processing-nlp-task-in-thai-nlp-ep-1/)

In [0]:
from fastai import *
from fastai.text import *
import sklearn.feature_extraction.text as sklearn_text
from matplotlib import pyplot as plt
import seaborn as sns

# 2. Dataset

ในเคสนี้เราจะใช้ IMDB Movie Review เป็นรีวิวหนังจากเว็บ IMDB ที่มีข้อความ และ คะแนนว่าเป็นแง่บวก หรือแง่ลบ

ในการพัฒนา เราจะใช้ Dataset ชุดเล็กก่อน จะได้เร็ว เมื่อเทสทุกอย่างเรียบร้อย แล้วจึงขยับไปใช้ Dataset ชุดเต็ม 

In [134]:
# IMDB Sample
path = untar_data(URLs.IMDB_SAMPLE)

path

PosixPath('/content/data/imdb_sample')

In [135]:
path.ls()

[PosixPath('/content/data/imdb_sample/texts.csv')]

In [0]:
# IMDB Sample
df = pd.read_csv(path/'texts.csv')
# df.head()

In [137]:
# IMDB_Sample
movie_reviews = (TextList.from_csv(path, "texts.csv", cols="text")
                        .split_from_df(col=2)
                        .label_from_df(cols=0))

len(movie_reviews.train.x), len(movie_reviews.valid.x)

(800, 200)

In [138]:
v = movie_reviews.vocab.itos
vocab_len = len(v)
vocab_len

6008

# 3. Preprocessing

## 3.1 Create Training N-Gram Matrix


ประกาศตัวแปร ที่จำเป็น

In [0]:
min_n = 1
max_n = 3

j_indices = []
indptr = []
values = []
indptr.append(0)
num_tokens = vocab_len

itongram = dict()
ngramtoi = dict()

In [0]:
for i, doc in enumerate(movie_reviews.train.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k+n]
            
            if str(ngram) not in ngramtoi: 
                if len(ngram) == 1: 
                    num = ngram[0]
                    ngramtoi[str(ngram)] = num
                    itongram[num] = ngram
                else:
                    ngramtoi[str(ngram)] = num_tokens
                    itongram[num_tokens] = ngram
                    num_tokens += 1
            this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))


In [0]:
len(values), len(j_indices), len(indptr) - 1, len(ngramtoi)

In [0]:
train_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr), 
                                                 shape=(len(indptr) - 1, len(ngramtoi)), 
                                                 dtype=int)

In [0]:
train_ngram_doc_matrix

In [0]:
len(ngramtoi), len(itongram)

ลองดูใน N-Gram Term-Document Matrix ที่ได้

### ตัวอย่าง 1

In [0]:
itongram[20005]

In [0]:
ngramtoi[str(np.array([15, 9, 710]))]

In [0]:
v[15], v[9], v[710]

### ตัวอย่าง 2

In [0]:
itongram[20050]

In [0]:
ngramtoi[str(np.array([14, 5, 2808]))]

In [0]:
v[14], v[5], v[2808]

### ตัวอย่าง 3

In [0]:
itongram[20080]

In [0]:
ngramtoi[str(np.array([23, 796,  11]))]

In [0]:
v[23], v[796], v[11]

## 3.2 Create Valid N-Gram Matrix

In [0]:
j_indices = []
indptr = []
values = []
indptr.append(0)

for i, doc in enumerate(movie_reviews.valid.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) in ngramtoi:
                this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

In [0]:
valid_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

In [0]:
valid_ngram_doc_matrix

In [0]:
train_ngram_doc_matrix

## 3.3 Save Data

In [0]:
scipy.sparse.save_npz("train_ngram_doc_matrix.npz", train_ngram_doc_matrix)

In [0]:
scipy.sparse.save_npz("valid_ngram_doc_matrix.npz", valid_ngram_doc_matrix)

In [0]:
with open('itongram.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('ngramtoi.pickle', 'wb') as handle: 
    pickle.dump(ngramtoi, handle, protocol=pickle.HIGHEST_PROTOCOL)

## 3.4 Load Data

In [0]:
train_ngram_doc_matrix = scipy.sparse.load_npz("train_ngram_doc_matrix.npz")
valid_ngram_doc_matrix = scipy.sparse.load_npz("valid_ngram_doc_matrix.npz")

In [0]:
with open("itongram.pickle", "rb") as handle:
    itongram = pickle.load(handle)
with open("ngramtoi.pickle", "rb") as handle:
    ngramtoi = pickle.load(handle)

ตรวจเช็คข้อมูล

In [0]:
train_ngram_doc_matrix, valid_ngram_doc_matrix

In [0]:
len(itongram), len(ngramtoi)

# 4. Predict

## 4.1 Naives Bayes

In [0]:
x = train_ngram_doc_matrix
y = movie_reviews.train.y

x.getnnz(), len(y)

In [0]:
positive = y.c2i['positive']
negative = y.c2i['negative']

In [0]:
x

k เอาไว้ limit จำนวนข้อมูล

In [0]:
k = 1000

In [0]:
pos = (y.items == positive)[:k]
neg = (y.items == negative)[:k]

In [0]:
xx = x[:k]

In [0]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [0]:
pr1 = (p1+1) / ((y.items == positive).sum() + 1)
pr0 = (p0+1) / ((y.items == negative).sum() + 1)

In [0]:
r = np.log(pr1/pr0)

In [0]:
b = np.log((y.items == positive).mean() / (y.items == negative).mean())
b

In [0]:
(y.items == positive).mean(), (y.items == negative).mean()

In [0]:
pre_preds = valid_ngram_doc_matrix @ r.T + b

In [0]:
pre_preds

In [0]:
preds = pre_preds.T > 0

In [0]:
preds[:10]

In [0]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [0]:
(preds == valid_labels).mean()

## 4.2 Binarized Naive Bayes

In [0]:
trn_x_ngram_sgn = train_ngram_doc_matrix.sign()
val_x_ngram_sgn = valid_ngram_doc_matrix.sign()

In [0]:
xx = trn_x_ngram_sgn[:k]

In [0]:
p0 = np.squeeze(np.array(xx[neg].sum(0)))
p1 = np.squeeze(np.array(xx[pos].sum(0)))

In [0]:
pr1 = (p1+1) / ((y.items == positive).sum() + 1)
pr0 = (p0+1) / ((y.items == negative).sum() + 1)

In [0]:
r = np.log(pr1/pr0)

In [0]:
b = np.log((y.items == positive).mean() / (y.items == negative).mean())
b

In [0]:
(y.items == positive).mean(), (y.items == negative).mean()

In [0]:
pre_preds = valid_ngram_doc_matrix @ r.T + b

In [0]:
pre_preds

In [0]:
preds = pre_preds.T > 0

In [0]:
preds[:10]

In [0]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [0]:
(preds == valid_labels).mean()

## 4.3 Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression

ใช้ CountVectorizer

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
veczr = CountVectorizer(ngram_range=(1, 3), preprocessor=noop, tokenizer=noop, max_features=800000)

In [0]:
docs = movie_reviews.train.x

In [0]:
train_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.train.x]
valid_words = [[docs.vocab.itos[o] for o in doc.data] for doc in movie_reviews.valid.x]

len(train_words), len(valid_words)

In [0]:
%%time
train_ngram_doc = veczr.fit_transform(train_words)

In [0]:
train_ngram_doc

ดูตัวอย่าง vocab Dictionary ที่ได้ เป็น N-Gram

In [0]:
from itertools import *

len(veczr.vocabulary_), list(islice(veczr.vocabulary_.items(), 20)), list(islice(veczr.vocabulary_.items(), 26000, 26020))

Transform ข้อมูล Validation Set ด้วย vocab Dictionary ของ Training Set

In [0]:
val_ngram_doc = veczr.transform(valid_words)
val_ngram_doc

In [0]:
vocab = veczr.get_feature_names()
len(vocab)

In [0]:
vocab[200000:200010]

In [0]:
y = movie_reviews.train.y

In [0]:
m = LogisticRegression(C=0.1, dual=True, solver='liblinear', max_iter=10000)
m.fit(train_ngram_doc, y.items);

preds = m.predict(val_ngram_doc)
(preds.T == valid_labels).mean()

## 4.4 Binarized Logistic Regression

In [0]:
m = LogisticRegression(C=0.1, dual=True, solver='liblinear', max_iter=10000)
m.fit(train_ngram_doc.sign(), y.items);

preds = m.predict(val_ngram_doc)
(preds.T == valid_labels).mean()

## 4.3/2 Logistic Regression กับ N-Gram ที่เราสร้างเอง

In [0]:
m2 = LogisticRegression(C=0.1, dual=True, solver='liblinear', max_iter=10000)
m2.fit(train_ngram_doc_matrix, y.items);

preds = m2.predict(valid_ngram_doc_matrix)
(preds.T == valid_labels).mean()

## 4.4/2 Binarized Logistic Regression กับ N-Gram ที่เราสร้างเอง

In [0]:
m2 = LogisticRegression(C=0.1, dual=True, solver='liblinear', max_iter=10000)
m2.fit(trn_x_ngram_sgn, y.items);

preds = m2.predict(val_x_ngram_sgn)
(preds.T == valid_labels).mean()

## 4.5 Logistic Regression กับ Log Count

เราจะ fit logistic regression ด้วย Regularized feature trigrams  log-count ratios.

In [0]:
x = train_ngram_doc_matrix.sign()
val_x = valid_ngram_doc_matrix.sign()
y = movie_reviews.train.y

In [0]:
positive = y.c2i['positive']
negative = y.c2i['negative']

In [0]:
k = 100000

In [0]:
pos = (y.items == positive)[:k]
neg = (y.items == negative)[:k]

In [0]:
xx = x[:k]

In [0]:
valid_labels = [o == positive for o in movie_reviews.valid.y.items]

In [0]:
p0 = np.squeeze(np.array(xx[neg]).sum(0))
p1 = np.squeeze(np.array(xx[pos]).sum(0))

In [0]:
pr0 = (pr0 + 1) / ((y.items == negative).sum() + 1)
pr0 = (pr1 + 1) / ((y.items == positive).sum() + 1)

In [0]:
r = np.log(pr1/pr0)
r.shape, r

In [0]:
np.exp(r)

In [0]:
b = np.log((y.items==positive).mean() / (y.items==negative).mean())
b

เราจะ Normalized ด้วย r ก่อน Fit Logistic regression

In [0]:
xx.shape, r.shape, r.reshape(-1, 1).shape

In [0]:
x_nb = xx.dot(r.reshape(-1, 1))
x_nb.shape

In [0]:
m = LogisticRegression(C=0.1, dual=True, solver='liblinear', max_iter=10000)
m.fit(x_nb, y.items)

In [0]:
val_x_nb = val_x.dot(r.reshape(-1, 1))
preds = m.predict(val_x_nb)
(preds.T == valid_labels).mean()

# สรุป

* เราได้เรียนรู้

# Credit

* https://www.youtube.com/watch?v=dt7sArnLo1g&list=PLtmWHNX-gukKocXQOkQjuVxglSDYWsSh9&index=6&t=0s
* https://www.bualabs.com/archives/3000/sentiment-classification-imdb-movie-reviews-with-naive-bayes-logistic-regression-nlp-ep-5/
* https://www.bualabs.com/archives/926/sentiment-analysis-imdb-movie-review-ulmfit-sentiment-analysis-ep-1/
* https://docs.fast.ai/text.data.html
* https://docs.python.org/2/library/collections.html
* https://en.wikipedia.org/wiki/Naive_Bayes_classifier
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
* https://en.wikipedia.org/wiki/N-gram
* https://web.stanford.edu/~jurafsky/slp3/3.pdf